In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from pyspark.ml.classification import  DecisionTreeClassifier
from sklearn.metrics import classification_report
import timeit
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from pyspark.sql import SparkSession
import findspark
import os
from pyspark.sql.functions import col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import math
from pyspark.ml.tuning import CrossValidator
from random import random
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import NaiveBayes


os.environ["JAVA_HOME"] = 'C:/Program Files/Java/jre1.8.0_333/'
os.environ["SPARK_HOME"] = 'C:/spark/spark-3.3.0-bin-hadoop2/'
os.environ["HADOOP_HOME"] = 'C:/hadoop/'
#os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local pyspark-shell'

In [19]:
findspark.find()

'C:/spark/spark-3.3.0-bin-hadoop2/'

In [20]:
findspark.init()

In [35]:
spark = SparkSession\
.builder\
.master("local")\
.config("spark.executor.cores", "4")\
.config("spark.executor.memory", "10g")\
.config("spark.driver.memory", "10g")\
.config("spark.executor.memoryOverhead", "10g")\
.appName("Churn Project").getOrCreate()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [36]:
df = spark.read.csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/ChurnModelScaled.csv',inferSchema=True,header=True)
scaled_df = spark.read.csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/ChurnModelScaled.csv',inferSchema=True,header=True)

In [37]:
df = df.drop('_c0')
scaled_df = scaled_df.drop('_c0')

In [38]:
numericCols = ['MonthNumber', 'ClientProfileSummary', 'Gender', 'CashActive_YN',
       'L1DSBBetCount', 'L7DSBBetCount', 'L30DSBBetCount', 'L90DSBBetCount',
       'L1DDepositCount', 'L7DDepositCount', 'L30DDepositCount',
       'L90DDepositCount', 'L7DOtherSportsBetCount', 'L30DOtherSportsBetCount',
       'L90DOtherSportsBetCount', 'L90DUnsuccessfulDepositCount',
       'DaysSinceLastSBCashAPD', 'DaysSinceLastSBAPD', 'OlderMale40',
       'CustomerConcession90days',
       'L7DSBTurnover', 'US_SportsTurnover7D', 'L30DSBTurnover',
       'US_SportsTurnover30D', 'CustomerConcession30days', 'L90DSBTurnover',
       'L1DSBTurnover', 'US_SportsTurnover1D', 'L30DSBFreeBetsHandle',
       'US_SportsTurnover90D', 'L7DSBFreeBetsHandle']




assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
scaled_df = assembler.transform(scaled_df)

Train/Test

In [39]:
train,test = df.randomSplit([0.7,0.3],seed=42)
train_scaled,test_scaled = scaled_df.randomSplit([0.7,0.3],seed=42)

<h2> Single Decision Tree

In [31]:
%%time
from pyspark.ml import Pipeline

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="Active_Next30Days_Cash_YN",featuresCol="features")


# Train model.  This also runs the indexers.
model = dt.fit(train_scaled)

# Make predictions.
predictions = model.transform(test_scaled)

# Select example rows to display.
predictions.select("prediction", "Active_Next30Days_Cash_YN", "features")

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Active_Next30Days_Cash_YN", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print ("Test Error = %g" % round(1.0 - accuracy,4))
print ("Accuracy of Model = %g" % round(accuracy,4))

Test Error = 0.1719
Accuracy of Model = 0.8281
CPU times: total: 31.2 ms
Wall time: 1min 16s


<h2> Naive Bayes Classifers

In [66]:
%%time

# create the trainer and set its parameters
nb = NaiveBayes(labelCol='Active_Next30Days_Cash_YN',featuresCol='features', modelType="gaussian",smoothing=1.0)

# train the model
model = nb.fit(train_scaled)

# select example rows to display.
predictions = model.transform(test_scaled)
#predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="Active_Next30Days_Cash_YN", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7635204064623242
CPU times: total: 0 ns
Wall time: 27.3 s


In [65]:
%%time

# create the trainer and set its parameters
nb = NaiveBayes(labelCol='Active_Next30Days_Cash_YN',featuresCol='features', modelType="complement",smoothing=1.0)

# train the model
model = nb.fit(train_scaled)

# select example rows to display.
predictions = model.transform(test_scaled)
#predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="Active_Next30Days_Cash_YN", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.751511898491102
CPU times: total: 15.6 ms
Wall time: 16.3 s


In [64]:
%%time

# create the trainer and set its parameters
nb = NaiveBayes(labelCol='Active_Next30Days_Cash_YN',featuresCol='features', modelType="multinomial",smoothing=1.0)

# train the model
model = nb.fit(train_scaled)

# select example rows to display.
predictions = model.transform(test_scaled)
#predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="Active_Next30Days_Cash_YN", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7503117144628844
CPU times: total: 15.6 ms
Wall time: 17.9 s


###################################################################################################################################


<h3> Support Vector Machine

In [69]:
%%time

from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(labelCol='Active_Next30Days_Cash_YN',featuresCol='features',maxIter=300,aggregationDepth=50, regParam=0.01)

# Fit the model
lsvcModel = lsvc.fit(train_scaled)

# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))


# select example rows to display.
predictions = lsvcModel.transform(test)
#predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="Active_Next30Days_Cash_YN", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Coefficients: [0.13185678870753856,0.7589561392486014,0.1224468449946559,1.4615638831327482,-7.572733217473998,69.01882415541422,37.73086335708482,12.98736916342468,-17.331924827905404,16.5415200181224,4.105495676697325,7.880711656800446,38.1921952954533,29.057281513310446,26.062265121802625,-7.441674941188936,-1.789883321514026,-1.2965821055643578,0.08582183953130032,-0.034138110532546714,10.272750266253393,13.340589266271667,6.07614326394855,12.9168374776068,0.13951346070113327,-0.926492741160736,-7.432986498798965,1.448829957355958,-21.121447610192547,3.7112371017138526,46.279263953732666]
Intercept: -1.2014786089400225
Test set accuracy = 0.7933383118744874
CPU times: total: 46.9 ms
Wall time: 33 s
